This Notebook will present the results of the Exploratory Data Analysis

# Clone Repository

In [8]:
# Executes the cell in bash mode
%%bash

if [ ! -d "/content/deep-deblurring/" ]; 
    then 
        git clone https://github.com/ElPapi42/deep-deblurring;
        cd deep-deblurring/
    else 
        cd deep-deblurring/; 
        git pull; 
fi;

git checkout development
cd ..

Already up to date.
Branch 'development' set up to track remote branch 'development' from 'origin'.


Switched to a new branch 'development'


# Install Requirements

In [2]:
# Executes the cell in bash mode
%%bash

#pip uninstall -y tensorflow tensor2tensor tensorboard tensorboardcolab tensorflow-datasets tensorflow-estimator tensorflow-gan tensorflow-hub tensorflow-metadata tensorflow-privacy tensorflow-probability

pip install colab-env
pip install --upgrade grpcio

pip install -r "/content/deep-deblurring/requirements.txt"

cd deep-deblurring/
python setup.py install
cd ..

  Created wheel for colab-env: filename=colab_env-0.2.0-cp36-none-any.whl size=3838 sha256=3875a011957d3120a6a5641f3c97e07bf06540aa3d4c1499bfb63a196ef79931
  Stored in directory: /root/.cache/pip/wheels/81/9c/85/7ec1faca43d1a5df32f33f4e22d5ae91f301a5406779528ead
Successfully built colab-env
  Found existing installation: grpcio 1.27.2
    Uninstalling grpcio-1.27.2:
      Successfully uninstalled grpcio-1.27.2
  Found existing installation: grpcio 1.28.1
    Uninstalling grpcio-1.28.1:
      Successfully uninstalled grpcio-1.28.1
running install
running bdist_egg
running egg_info
creating deblurrer.egg-info
writing deblurrer.egg-info/PKG-INFO
writing dependency_links to deblurrer.egg-info/dependency_links.txt
writing requirements to deblurrer.egg-info/requires.txt
writing top-level names to deblurrer.egg-info/top_level.txt
writing manifest file 'deblurrer.egg-info/SOURCES.txt'
writing manifest file 'deblurrer.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg

zip_safe flag not set; analyzing archive contents...
deblurrer.scripts.datasets.__pycache__.compile_all.cpython-36: module references __file__
deblurrer.scripts.datasets.__pycache__.download.cpython-36: module references __file__
deblurrer.scripts.datasets.__pycache__.generate_csv.cpython-36: module references __file__
deblurrer.scripts.datasets.__pycache__.generate_dataset.cpython-36: module references __file__
deblurrer.scripts.datasets.__pycache__.generate_tfrecord.cpython-36: module references __file__
deblurrer.scripts.datasets.__pycache__.kaggle_blur.cpython-36: module references __file__
deblurrer.scripts.training.__pycache__.alt_train.cpython-36: module references __file__
deblurrer.scripts.training.__pycache__.test.cpython-36: module references __file__
deblurrer.scripts.training.__pycache__.train.cpython-36: module references __file__


In [0]:
# Add deblurrer package to path

import sys
sys.path += ['/content/deep-deblurring']

# Data Download

In [4]:
# Setup credentials from google drive

import colab_env

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
# Executes the cell in bash mode
%%bash

python /content/deep-deblurring/deblurrer/scripts/datasets/compile_all.py


Generating .csv files
.csv files successfully generated
Generating TFRecords
Generating train.csv TFRecords
Generating valid.csv TFRecords
Generating test.csv TFRecords
TFRecords succesfully generated


2020-04-08 19:07:35.556493: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
100%|██████████| 1.49G/1.49G [00:19<00:00, 84.1MB/s]2020-04-08 19:08:40.913328: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-08 19:08:40.958565: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-04-08 19:08:40.959566: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-04-08 19:08:40.959614: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-04

# Hyper-Parameters

In [0]:
import os

os.environ['EPOCHS'] = '30'
os.environ['BATCH_SIZE'] = '4'
os.environ['IMAGE_SIZE'] = '128'
os.environ['IMAGE_SIZE_MULTIPLIER'] = '2'
os.environ['FPN_CHANNELS'] = '128'
os.environ['DISC_FILTERS'] = '6'
os.environ['DISC_CONV_COUNT'] = '2'
os.environ['GEN_LR'] = '0.00001'
os.environ['DISC_LR'] = '0.00001'
os.environ['USE_MIXED_PRECISION'] = '0'

# Training

In [0]:
# Starts training

from deblurrer.scripts.training import alt_train as train

model, gen_opt, disc_opt, strategy = train.run(
    '/content/deep-deblurring/datasets/tfrecords',
    output_folder='/content/deep-deblurring/output',
    warm_epochs=20,
)

In [0]:
model, gen_opt, disc_opt, strategy = train.run(
    '/content/deep-deblurring/datasets/tfrecords',
    model,
    gen_opt,
    disc_opt,
    strategy,
    output_folder='/content/deep-deblurring/output'
)